# Accessibility to schools in rural areas

In [ ]:
# If using colab
# Takes around 2-3 min
!pip install git+https://github.com/CityScope/UrbanAccessAnalyzer.git
!pip install matplotlib mapclassify folium
!apt-get install -y osmium-tool

# Restart notebook after installing this if needed

In [ ]:
import os
from datetime import datetime, date, timedelta, time
import pandas as pd
import geopandas as gpd
import os

import osmnx as ox

import matplotlib.pyplot as plt

import UrbanAccessAnalyzer.isochrones as isochrones
import UrbanAccessAnalyzer.graph_processing as graph_processing
import UrbanAccessAnalyzer.osm as osm
import UrbanAccessAnalyzer.utils as utils
import UrbanAccessAnalyzer.raster_utils as raster_utils
import UrbanAccessAnalyzer.population as population

import zipfile
import numpy as np

## 1 Points of interest

Points of interest (poi): Schools as point geometry

Area of interest (aoi): Polygon to do the analisys

### 1.1 Results folder

In [ ]:
results_path = os.path.normpath("results")
os.makedirs(results_path,exist_ok=True)

### 1.2 Area of interest

Option 1: Write the city name

In [ ]:
city_name = "Arevalo, España"
city_filename = utils.sanitize_filename(city_name)
aoi = utils.get_city_geometry(city_name)
geo_suggestions = utils.get_geographic_suggestions_from_string(city_name,user_agent="app")
geo_suggestions

Option 2: Load your own file (.gpkg or .shp)

In [ ]:
# aoi = gpd.read_file("")
# city_name = ""

Use UTM coords and creat aoi_download with a buffer of X meters

In [ ]:
aoi = gpd.GeoDataFrame(geometry=[aoi.union_all()],crs=aoi.crs) # Ensure there is only one polygon
aoi = aoi.to_crs(aoi.estimate_utm_crs()) # Convert to utm
aoi_download = aoi.buffer(20000) # Area to do streets and poi requests

Map of your area of interest and the download area (aoi_buffer)

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m

### 1.3 OpenStreetMap

In [ ]:
query = """
[out:xml] [timeout:25];
(
    node["amenity"="school"]( {{bbox}});
    way["amenity"="school"]( {{bbox}});
    relation["amenity"="school"]( {{bbox}});
);
(._;>;);
out body;
"""

In [ ]:
# If it fails execute again

poi = osm.overpass_api_query(query,aoi_download)
poi.geometry = poi.geometry.centroid
poi = poi.to_crs(aoi.crs)
poi

Map with points of interest (schools), download area and area of interest

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m=poi.explore(m=m,color='blue')
m

## 2 Street graph

### 2.1 Regionwise file and cropping

- Download best regionwise pbf file. (Covers a large area)

- Crop it to cover our area of interest and save it in .osm format

In [ ]:
osm_xml_file = os.path.normpath(results_path+"/streets.osm")
streets_graph_path = os.path.normpath(results_path+"/streets.graphml")
streets_path = os.path.normpath(results_path+"/streets.gpkg")
level_of_service_streets_path = os.path.normpath(results_path+"/level_of_service_streets.gpkg")

In [ ]:
# Select what type of street network you want to load
network_filter = osm.osmium_network_filter("walk+bike+primary")
# Download the region pbf file crop it by aoi and convert to osm format
osm.geofabrik_to_osm(
    osm_xml_file,
    input_file=results_path,
    aoi=aoi_download,
    osmium_filter_args=network_filter,
    overwrite=False
)

### 2.2 Load to osmnx

This way the street network is a networkx graph

In [ ]:
# Load
G = ox.graph_from_xml(osm_xml_file)
# Project geometry coordinates to UTM system to allow euclidean meassurements in meters (sorry americans)
G = ox.project_graph(G,to_crs=aoi.estimate_utm_crs())
# Save the graph in graphml format to avoid the slow loading process
ox.save_graphml(G,streets_graph_path)

### 2.3 Simplify graph

Edges with length smaler than X meters are deleted and its nodes merged

In [ ]:
min_edge_length = 30

G = graph_processing.simplify_graph(G,min_edge_length=min_edge_length,min_edge_separation=min_edge_length*2,undirected=True)
# Save the result in graphml format
ox.save_graphml(G,streets_graph_path)

street_edges = ox.graph_to_gdfs(G,nodes=False)
street_edges.to_file(streets_path)

### 2.4 Points of interest

Add pois to street graph

In [ ]:
G, osmids = graph_processing.add_points_to_graph(
    poi,
    G,
    max_dist=100+min_edge_length, # Maximum distance from point to graph edge to project the point
    min_edge_length=min_edge_length # Minimum edge length after adding the new nodes
)
poi['osmid'] = osmids # Add the ids of the nodes in the graph to points

## 3 Compute isochrones

### 3.1 Distance steps

In [ ]:
distance_steps = [1000,3000,20000]
level_of_services = ['walk','bike','bus']

In [ ]:
level_of_service_graph = isochrones.graph(
    G,
    poi,
    distance_steps, # If service_quality_col is None it could be a list of distances
    service_quality_col = None, # If all points have the same quality this could be None
    level_of_services = level_of_services, # could be None and it will set to the sorted unique values of the matrix
    min_edge_length = min_edge_length # Do not add new nodes if there will be an edge with less than this length
)
# Save edges as gpkg
level_of_service_nodes, level_of_service_edges = ox.graph_to_gdfs(level_of_service_graph)
level_of_service_edges.to_file(level_of_service_streets_path)

#### Lets visualize the results on a map

In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
)

m = poi[[
    "name",
    "geometry"
]].explore(
    m=m,
    color="black",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)

m.save(results_path + "/level_of_service_streets.html")
# If map does not render
# import webbrowser
# webbrowser.open(results_path + "/PToffer_map.html")
m

## 4 Population

### 4.1 Download Worldpop tif file

- One file for every country
- 100m pixel size
- tif format
- available from 2000 to 2030
- gender and age

In [ ]:
population_file = population.download_worldpop_population(
    aoi_download,
    2025,
    folder=results_path,
    resolution="100m",
    dataset="age_structures",
    subset="U18"
)

In [ ]:
zip_path = population_file

# Extract to the same directory as the zip file
extract_dir = os.path.splitext(zip_path)[0]
os.makedirs(extract_dir, exist_ok=True)

# Decompress the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Find the .tif file that contains '_T_'
for file_name in os.listdir(extract_dir):
    if file_name.lower().endswith('.tif') and '_T_' in file_name:
        population_file = os.path.join(extract_dir, file_name)
        break
else:
    raise FileNotFoundError("No .tif file containing '_T_' found in the zip archive.")

### 4.2 Filter population grid by streets

The population files are not very precise in location. Sometimes population appears in inaccessible places. This makes the results slightly better.

In [ ]:
street_buffer = 25

pop_raster, transform, crs = population.filter_population_by_streets(
    streets_gdf=street_edges,
    population=population_file,
    street_buffer=street_buffer,
    aoi=aoi_download,
    min_population=1
)

### 4.3 Assign level of service to each population cell

In [ ]:
level_of_service_polygons = level_of_service_edges.to_crs(level_of_service_edges.estimate_utm_crs()).copy()
level_of_service_polygons.geometry = level_of_service_polygons.geometry.simplify(street_buffer/2).buffer(street_buffer,resolution=4)
level_of_service_raster = raster_utils.rasterize(
    gdf=level_of_service_polygons,
    shape=pop_raster,
    transform=transform,
    crs=crs,
    value_column='level_of_service',
    value_order=level_of_services
)

pop_gdf = raster_utils.vectorize(pop_raster,transform,crs,keep_nodata=True,nodata=0,min_value=1)
pop_gdf = pop_gdf.rename(columns={'value':'population'})
pop_gdf['population'] = pop_gdf['population'].astype(float).fillna(0)
pop_gdf['level_of_service'] = level_of_service_raster.flatten()
pop_gdf = pop_gdf[['id','population', *pop_gdf.columns[3:],'geometry']]
pop_gdf = pop_gdf[pop_gdf['population'] > 1].reset_index(drop=True)
pop_gdf = pop_gdf[pop_gdf.intersects(aoi.to_crs(pop_gdf.crs).union_all())]

level_of_service_edges = level_of_service_edges[level_of_service_edges.intersects(aoi.to_crs(level_of_service_edges.crs).union_all())]
level_of_service_nodes = level_of_service_nodes[level_of_service_nodes.intersects(aoi.to_crs(level_of_service_nodes.crs).union_all())]
poi = poi[poi.intersects(aoi.to_crs(poi.crs).union_all())]

pop_gdf.to_file(os.path.normpath(results_path + "/population.gpkg"))

level_of_service_edges.to_file(level_of_service_streets_path)


In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
)

m = poi[[
    "name",
    "geometry"
]].explore(
    m=m,
    color="black",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)

pop_gdf_points = pop_gdf.copy()
pop_gdf_points.geometry = pop_gdf_points.geometry.centroid
m=pop_gdf_points.explore(
    m=m,
    column="level_of_service",            # color by service level
    cmap="viridis",
    legend=True,
    tooltip=["population", "level_of_service"],
    marker_type="circle_marker",
    style_kwds={
        # define a dynamic radius for each feature
        "style_function": lambda x: {
            "radius": (np.log1p(x["properties"]["population"]) /
                       np.log1p(pop_gdf_points["population"].max()) * 5),
            "color": "black",      # border color
            "weight": 1,           # border thickness
            "opacity": 1.0,        # border opacity
            "fillOpacity": 1.0,    # fill opacity
        },
    },
)

m.save(results_path + "/level_of_service_population.html")
# If map does not render
# import webbrowser
# webbrowser.open(results_path + "/PToffer_map.html")
m

Important files:

- streets.gpkg Has the street geometry as lines (all streets)
- level_of_service_streets.gpkg Has the street geometry as lines with the level of service (only streets with level of service)
- population.gpkg Is a grid with population and level of service